# 9. Obtendo Dados

### stdin e stdout

* Se os scripts de Python são executados via linha de comando, é possível canalizar (pipe) os dados usando sys.stdin e sys.stdout.

* Abaixo um script que lê linhas de um texto e devolve as que combinarem como uma expressão regular:

~~~ Python
# Salvo como egrep.py
import sys, re

# sys.argv é a lista dos argumentos da linha de comandos
# sys.argv [0] é o nome do programa em si
# sys.argv [1] será o regex especificado na linha de comandos
regex = sys.argv[1]

# para cada linha passada pelo script
for line in sys.stdin:
    # se combinar com o regex, escreva-o para o stdout
    if re.search(regex, line):
        sys.stdout.write(line)
~~~

* O próximo conta as linhas recebidas e exibe a contagem:

~~~ Python
# Salvo como line_count.py
import sys

count = 0
for line in sys.stdin:
    count += 1
    
# print vai para sys.stdout
print(count)
~~~

* E poderia usá-los utilizando o seguinte comando:

In [1]:
cat Dados/File.txt | python Codigos/egrep.py "[0-9]" | python Codigos/line_count.py

3


* O caractere | (pipe) significa "use a saída do comando da esquerda como entrada do comando da direita"
* Permite a construção de pipelines elaborados de processamento de dados.

* O script abaixo conta as palavras e exibe as mais comuns:

~~~ Python
# Salvo como most_commom_words.py
import sys
from collections import Counter

# passa o número de palavras como primeiro argumento
try:
    num_words = int(sys.argv[1])
except:
    print ()
    sys.exit(1) # código de saída não-zero indica erro
    
counter = Counter(word.lower() # palavras em minúsculas
                  for line in sys.stdin #
                  for word in line.strip().split() # se separam por espaços
                  if word)

# pula as 'palavras' vazias
for word, count in counter.most_common(num_words):
    sys.stdout.write(str(count))"usage: most_common_words.py num_words"
    sys.stdout.write("\t")
    sys.stdout.write(word)
    sys.stdout.write("\n")
    
~~~

* E finalmente pode ser chamado assim:

In [2]:
cat Dados/some_text.txt | python Codigos/most_common_words.py 5

10	and
10	to
8	the
7	of
4	in


### Lendo Arquivos

* É possível ler a partir de e escrever nos arquivos através do código python.

#### O básico de arquivos texto

* O primeiro passo é obter um objeto de arquivo usando *open*:

~~~ Python
# 'r' significa somente leitura
file_for_reading = open('reading_file.txt', 'r')

# 'w' é escrever - - destruirá o arquivo se ele já existir!
file_for_writing = open('writing_file.txt', 'w')

# 'a' é anexar - - para adicionar ao final do arquivo
file_for_appending = open('appending_file.txt', 'a')

# não se esqueça de fechar os arquivos ao terminar
file_for_writing.close()
~~~

* O *open* deve ser usado em um bloco *with*, pois fecha automaticamente os arquivos:

~~~ Python
with open(filename,'r') as f:
    data = function_that_gets_data_from(f)
    
# neste ponto f já foi fechado, não tente usá-lo
process(data)
~~~

* Se você precisar ler o arquivo inteiro, pode-se iterar sobre as linhas do arquivo usando for:

~~~ Python
starts_with_hash = 0

with open('input.txt','r') as f:
    for line in file: # observe cada linha do arquivo
        if re.match("^#",line): # use um regex para ver se começa com '#'
            starts_with_hash += 1 # se começar, adicione 1 à contagem
~~~

* Toda linha obtida dessa forma termina com um caractere '\n' (new line), frequentemente iremos removê-lo.

#### Arquivos delimitados

* Com frequência você trabalhará com arquivos com muitos dados separados por vírgula (comma-separated) ou por tabulação (tab-separated).
   * Cada linha possui diversos campos com virgula ou tabulação inidicando o fim de um campo.
   
   
* Se tivéssemos um arquivo de preçoes de ações delimitado por tabulações:

In [3]:
cat Dados/tab_delimited_stock_prices.txt

6/20/2014	AAPL	90.91
6/20/2014	MSFT	41.68
6/20/2014	FB	64.5
6/19/2014	AAPL	91.86
6/19/2014	MSFT	41.51
6/19/2014	FB	64.34

* Podemos processá-los com:

In [4]:
import csv

with open('Dados/tab_delimited_stock_prices.txt','r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        date = row[0]
        symbol = row[1]
        closing_price = float(row[2])
        print(date, symbol, closing_price)

6/20/2014 AAPL 90.91
6/20/2014 MSFT 41.68
6/20/2014 FB 64.5
6/19/2014 AAPL 91.86
6/19/2014 MSFT 41.51
6/19/2014 FB 64.34


* Se o arquivo possui cabeçalho você pode pular a linha com reader.next() ou obter cada linha como um dicionário com csv.DictReader().

In [5]:
cat Dados/colon_delimited_stock_prices.txt

date:symbol:closing_price
6/20/2014:AAPL:90.91
6/20/2014:MSFT:41.68
6/20/2014:FB:64.5

In [6]:
with open('Dados/colon_delimited_stock_prices.txt', 'r') as f:
    reader = csv.DictReader(f, delimiter=':')
    for row in reader:
        date = row["date"]
        symbol = row["symbol"]
        closing_price = float(row["closing_price"])
        print(date, symbol, closing_price)

6/20/2014 AAPL 90.91
6/20/2014 MSFT 41.68
6/20/2014 FB 64.5


* Se o arquivo não tiver cabeçalhos você ainda pode usar passar a chave a eles como o parâmetro fieldnames.
* Você pode escrever os dados delimitados usando csv.writer()

In [7]:
today_prices = { 'AAPL' : 90.91, 'MSFT' : 41.68, 'FB' : 64.5 }

with open('Dados/comma_delimited_stock_prices.txt','w') as f:
    writer = csv.writer(f, delimiter=',')
    for stock, price in today_prices.items():
        writer.writerow([stock, price])

In [8]:
cat Dados/comma_delimited_stock_prices.txt

AAPL,90.91
MSFT,41.68
FB,64.5


* Uma forma errada de fazer isso seria:

In [9]:
results = [["test1", "success", "Monday"],
           ["test2", "success, kind of", "Tuesday"],
           ["test3", "failure, kind of", "Wednesday"],
           ["test4", "failure, utter", "Thursday"]]

# não faça isso!
with open('Dados/bad_csv.txt', 'w') as f:
    for row in results:
        f.write(",".join(map(str, row))) # talvez tenha muitas vírgulas nele!
        f.write("\n") # a linha pode ter newlines também!

In [10]:
cat Dados/bad_csv.txt

test1,success,Monday
test2,success, kind of,Tuesday
test3,failure, kind of,Wednesday
test4,failure, utter,Thursday


### Extraindo Dados da Internet

* Outra maneira de obter dados é extraindo de páginas web.
* Não é uma tarefa tão simples.

#### HTML e sua subsequente pesquisa

* HTML não é bem formulado, muito menos comentado. Necessitamos de ajuda para entender tudo isso.
* Para extrair esses dados utilizamos a biblioteca BeautifulSoup.
  * Constrói uma ávore a partir de elementos da página.
  * Fornece interface simples para acessá-los.
  
  
* Usaremos a biblioteca requests para a comunicação com o HTTP.
* E utilizaremos a biblioteca html5lib como interpretador HTML.
* Abaixo, um exemplo de uso das bibliotecas citadas, após isso, serão usados alguns métodos simples.

In [11]:
from bs4 import BeautifulSoup
import requests

html = requests.get("http://www.example.com").text
soup = BeautifulSoup(html, 'html5lib')

In [12]:
soup

<!DOCTYPE html>
<html><head>
    <title>Example Domain</title>

    <meta charset="utf-8"/>
    <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
    <meta content="width=device-width, initial-scale=1" name="viewport"/>
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>    
</head>

<body>
<div>
    <h1>Example Domain</h1>
    <p>This domain is

* Depois, serão trabalhados utilizando as tags de marcações do HTML.
* Veja o exemplo abaixo que pega elementos da tag p:

In [13]:
first_paragraph = soup.find('p')
# ou
first_paragraph_alt = soup.p

In [14]:
print(first_paragraph)
print()
print(first_paragraph_alt)

<p>This domain is for use in illustrative examples in documents. You may use this
    domain in literature without prior coordination or asking for permission.</p>

<p>This domain is for use in illustrative examples in documents. You may use this
    domain in literature without prior coordination or asking for permission.</p>


* É possível obter o conteúdo da tag usando a propriedade text:

In [15]:
first_paragraph_text = soup.p.text
first_paragraph_words = soup.p.text.split()

In [16]:
print(first_paragraph_text)
print()
print(first_paragraph_words)

This domain is for use in illustrative examples in documents. You may use this
    domain in literature without prior coordination or asking for permission.

['This', 'domain', 'is', 'for', 'use', 'in', 'illustrative', 'examples', 'in', 'documents.', 'You', 'may', 'use', 'this', 'domain', 'in', 'literature', 'without', 'prior', 'coordination', 'or', 'asking', 'for', 'permission.']


* Podemos extrair os atributos de uma marcação tratando como um dict: 

In [17]:
first_paragraph_id = soup.p['id'] # retorna um KeyError se não tiver 'id'

KeyError: 'id'

In [18]:
first_paragraph_id2 = soup.p.get('id') # retorna None se não tiver 'id'
print(first_paragraph_id2)

None


* Pode-se obter múltiplas marcações ao mesmo tempo:

In [19]:
all_paragraphs = soup.find_all('p') # ou apenas soup('p')
paragraphs_with_ids = [p for p in soup('p') if p.get('id')]

* Também podemos encontrar marcações com uma *class* específica:

In [20]:
important_paragraphs = soup('p', {'class' : 'important'})
important_paragraphs2 = soup('p', 'important')
important_paragraphs3 = [p for p in soup('p')
                         if 'important' in p.get('class', [])]

* Essas funcionalidades nos possibilitarão fazer algumas coisas.
* Para aplicações mais complicadas, consulte a documentação.

### Usando APIs

* Muitos websites e serviços web fornecem APIs que permitem a solicitação de dados estruturados.

#### JSON (e XML)

* O HTTP é um protocolo que permite a transferência de texto.
* Os dados que você solicita por uma API devem ser serializados por meio de strings.
* Geralmente essa serialização é feita através de JSON.
* Pela sua estrutura parecer muito com dicts python, facilita a interpretação das strings:

In [21]:
{"title" : "Data Science Book",
 "author" : "Joel Grus",
 "publicationYear" : 2014,
 "topics" : [ "data", "science", "data science"]}

{'title': 'Data Science Book',
 'author': 'Joel Grus',
 'publicationYear': 2014,
 'topics': ['data', 'science', 'data science']}

* Podemos analisar JSON utilizando o módulo json do Python.
* Usaremos a função loads para desserializar um objeto JSON:

In [22]:
import json
serialized = """{ "title" : "Data Science Book", "author" : "Joel Grus", "publicationYear" : 2014,
"topics" : [ "data", "science", "data science"] }"""

# analisa o json para criar um dict do Python
deserialized = json.loads(serialized)
if "data science" in deserialized["topics"]:
    print (deserialized)

{'title': 'Data Science Book', 'author': 'Joel Grus', 'publicationYear': 2014, 'topics': ['data', 'science', 'data science']}


* As vezes a API fornece apenas respostas em XML. 
* Para tal pode-se usar o módulo BeautifulSoup para obter os dados.

#### Usando uma API não autenticada

* A maioria das APIs requerem uma autenticação de quem deseja usar. 
* Daremos uma olhada na API do github para praticar acessos não autenticados:

In [23]:
import requests, json
endpoint = "https://api.github.com/users/acucenarodrigues1998/repos"
repos = json.loads(requests.get(endpoint).text)

* repos é uma list de dicts do Python, cada uma representando um repositorio público no meu github.
* Com esses dados, é possível descobrir em quais meses e dias eu possuo mais tendência a criar repositórios.
* As datas na resposta são strings.
* Usaremos o módulo dateutil para analisar as datas.
* Usaremos a função dateutil.parser.parse.

In [30]:
from dateutil.parser import parse
from collections import Counter

dates = [parse(repo["created_at"]) for repo in repos]
month_counts = Counter(date.month for date in dates)
weekday_counts = Counter(date.weekday() for date in dates)

print("Meses com mais repositórios criados:")
print(list(month_counts.keys()))
print("Dias com mais repositórios criados:")
print(list(weekday_counts.keys()))

Meses com mais repositórios criados:
[12, 2, 10, 8, 9, 11]
Dias com mais repositórios criados:
[0, 2, 1, 3, 4]


* Podemos pegar as linguagens dos 5 últimos repos criados:

In [31]:
last_5_repositories = sorted(repos, key=lambda r: r["created_at"], reverse=True)[:5]
last_5_languages = [repo["language"] for repo in last_5_repositories]
print(last_5_languages)

['Jupyter Notebook', 'Jupyter Notebook', 'Python', 'Java', 'Jupyter Notebook']


* Existem bibliotecas que omitem o baixo nível do acesso a APIs, como feito acima. Elas podem poupar muito trabalho.

### Exemplo: Usando as APIs do Twitter

* O Twitter é uma grande fonte de dados para trabalhar.
* Você pode usá-la para praticamente tudo que imaginar, desde que possua o acesso aos dados.
* Esse acesso pode ser obtido através de uma API.
* Para interagir com a API, usaremos a biblioteca Twython.

#### Obtendo Credenciais

* Para usar a API do Twitter são necessárias algumas credenciais.
* Para obtê-las, é necessário acessar o site https://apps.twitter.com, criar uma conta de desenvolvedor e em seguida criar um app.
  * Não colocarei o passo-a-passo, pois, são frequentes as mudanças nesse processo.
  
  
* Após a criação do APP, são geradas a chave e o segredo do consumidor que servem para identificar qual APP está acessando os dados

#### Usando Twython

* Utilizaremos o Search API que necessita apenas da chave e do segredo do consumidor.

In [37]:
from twython import Twython

CONSUMER_KEY = 
CONSUMER_SECRET = 
ACCESS_TOKEN = 
ACCESS_TOKEN_SECRET = 

twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET)

# Busca por tweets contendo a frase "data science"
for status in twitter.search(q='"data science"')["statuses"]:
    user = status["user"]["screen_name"].encode('utf-8')
    text = status["text"].encode('utf-8')
    print("{}: {}".format(user, text))
    print()

b'_SelloM': b'No ways. Code is profound. Maybe data science or computational thinking but / data / digital literacy but coding at\xe2\x80\xa6 https://t.co/nDxKWEtWxr'

b'elfprince13': b'RT @genevievemp: So many statistical methods were invented by farmers for farming. It\xe2\x80\x99s the first application area for \xe2\x80\x98data science\xe2\x80\x99.\n1/2'

b'Anna_LisaRob': b'RT @NickBiniker: Higher education is not at its best when it is facing inward, but outward - @mikekrauseTN Congratulations @capigian and te\xe2\x80\xa6'

b'grjenkin': b'The top 3 mistakes companies make when adopting AI: Amazon head of data science at AWS machine learning solutions l\xe2\x80\xa6 https://t.co/QIx2xutHFm'

b'capgemini_ch': b"RT @Capgemini: We're pleased to announce today the acquisition of Advectas, a leading business intelligence and data science company in Sca\xe2\x80\xa6"

b'Adam_IT_': b'RT @PythonDevelop: Python For Data Science - How to use Data Science with Python\n\n\xe2\x98\x9e https://t.co

* Isso não é muito interessante. O Search API mostra apenas a parte dos resultados que ele quer.
* Para mais dados, é necessário utilizar o Streaming API.

In [38]:
from twython import TwythonStreamer

# Anexando dados a variável global para simplificar o exemplo.
tweets = []

class MyStreamer(TwythonStreamer):
    
    def on_success(self, data):
        # O que fazer quando o twitter manda dados.
        # Os dados serão dicts de python representando um twitter.
        
        # Coletar dados apenas em inglês
        if data['lang'] == 'en':
            tweets.append(data)
            print("Received tweet #{}".format(len(tweets)))
            
        if len(tweets) >= 1000:
            self.disconnect()
            
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

* MyStreamer vai se connectar com o stream do Twitter e esperar que o twitter o abasteça de dados.
* Ao receber dados, ele passa para on_success() que anexa a lista de tweets se a lingua for inglesa e desconecta após obter 1000 tweets.
* Agora vamos testá-lo:

In [39]:
stream = MyStreamer(CONSUMER_KEY, CONSUMER_SECRET,
                   ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# Começa a consumir status públicos que contenham a palavra chave "data"
stream.statuses.filter(track='data')

# Se quisessemos começar a consumir uma amostra dos dados coletados
stream.statuses.sample()

Received tweet #1
Received tweet #2
Received tweet #3
Received tweet #4
Received tweet #5
Received tweet #6
Received tweet #7
Received tweet #8
Received tweet #9
Received tweet #10
Received tweet #11
Received tweet #12
Received tweet #13
Received tweet #14
Received tweet #15
Received tweet #16
Received tweet #17
Received tweet #18
Received tweet #19
Received tweet #20
Received tweet #21
Received tweet #22
Received tweet #23
Received tweet #24
Received tweet #25
Received tweet #26
Received tweet #27
Received tweet #28
Received tweet #29
Received tweet #30
Received tweet #31
Received tweet #32
Received tweet #33
Received tweet #34
Received tweet #35
Received tweet #36
Received tweet #37
Received tweet #38
Received tweet #39
Received tweet #40
Received tweet #41
Received tweet #42
Received tweet #43
Received tweet #44
Received tweet #45
Received tweet #46
Received tweet #47
Received tweet #48
Received tweet #49
Received tweet #50
Received tweet #51
Received tweet #52
Received tweet #53
Re

* Com os dados encontrados é possível, por exemplo, encontrar as hashtags mais famosas:

In [40]:
top_hashtags = Counter(hashtag['text'].lower()
                      for tweet in tweets
                      for hashtag in tweet["entities"]["hashtags"])

print(top_hashtags.most_common(5))

[('data', 29), ('ai', 21), ('blockchain', 17), ('fintech', 17), ('bitcoin', 16)]
